In [123]:
import cwru                      # Case Western Reserve University Bearing Data
import pandas as pd              # datafrme
import numpy as np               # array
from collections import Counter  # Class 갯수 세기
import matplotlib.pyplot as plt  # Visualization
%matplotlib inline
from scipy.fftpack import fft, ifft
from acoustics.cepstrum import complex_cepstrum, real_cepstrum
from sklearn.preprocessing import scale
from scipy.spatial import distance
from numpy.linalg import inv

In [285]:
cwru_data = cwru.CWRU("12DriveEndFault", "1797", 12000)

In [286]:
x_train = np.asarray(cwru_data.X_train)
x_test = np.asarray(cwru_data.X_test)

y_train = np.asarray(cwru_data.y_train)
y_test = np.asarray(cwru_data.y_test)

In [311]:
normal_test_idx

array([16, 23, 25, 40, 41])

In [312]:
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1])

In [287]:
print(Counter(y_train)) # Train 데이터 클래스 분포
print(Counter(y_test))  # Test 데이터 클래스 분포

Counter({15: 15, 0: 7, 1: 7, 2: 7, 3: 7, 4: 7, 5: 7, 6: 7, 7: 7, 8: 7, 9: 7, 10: 7, 11: 7, 12: 7, 13: 7, 14: 7})
Counter({15: 5, 0: 3, 1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3, 9: 3, 10: 3, 11: 3, 12: 3, 13: 3, 14: 3})


In [288]:
# Normal 클래스
normal_train_idx, = np.where(y_train == 15)   
normal_test_idx, = np.where(y_test == 15)     

# Abnormal 클래스
abnormal_train_idx, = np.where(y_train != 15)
abnormal_test_idx, = np.where(y_test != 15)

In [289]:
# Train 데이터 클래스 수정
for i in range(len(y_train)):
    if i in normal_train_idx:
        y_train[i] = 0 # Normal 클래스
    else:
        y_train[i] = 1 # Abnormal 클래스    
        
# Test 데이터 클래스 수정
for i in range(len(y_test)):
    if i in normal_test_idx:
        y_test[i] = 0 # Normal 클래스
    else:
        y_test[i] = 1 # Abnormal 클래스  

In [290]:
def do_FFT(signal_set):
    window = np.hamming(12000)
    fft_rlt = fft(window * signal_set)
    return(fft_rlt)

def do_Cepstrum(spectrum):
    cep_result = ifft(np.log(spectrum**2))**2
    return(cep_result)

    #ifft(np.log(np.abs(spectrum)))

In [291]:
#Data split
normal_x_train = pd.DataFrame(x_train[normal_train_idx])
normal_x_test = pd.DataFrame(x_test[normal_test_idx])

abnormal_x_train = pd.DataFrame(x_train[abnormal_train_idx])
abnormal_x_test = pd.DataFrame(x_test[abnormal_test_idx])

In [292]:
#Standardization
normal_x_train_scaled = pd.DataFrame(scale(normal_x_train, axis = 1))
normal_x_test_scaled = pd.DataFrame(scale(normal_x_test, axis = 1))

abnormal_x_train_scaled = pd.DataFrame(scale(abnormal_x_train, axis = 1))
abnormal_x_test_scaled = pd.DataFrame(scale(abnormal_x_test, axis = 1))

In [293]:
#Hamming and FFT
normal_x_train_fft = np.apply_along_axis(do_FFT, 1, normal_x_train_scaled)
normal_x_test_fft = np.apply_along_axis(do_FFT, 1, normal_x_test_scaled)

abnormal_x_train_fft = np.apply_along_axis(do_FFT, 1, abnormal_x_train_scaled)
abnormal_x_test_fft = np.apply_along_axis(do_FFT, 1, abnormal_x_test_scaled)

In [294]:
#Cepstrum
normal_x_train_cep = np.apply_along_axis(do_Cepstrum, 1, normal_x_train_fft)
normal_x_test_cep = np.apply_along_axis(do_Cepstrum, 1, normal_x_test_fft)

abnormal_x_train_cep = np.apply_along_axis(do_Cepstrum, 1, abnormal_x_train_fft)
abnormal_x_test_cep = np.apply_along_axis(do_Cepstrum, 1, abnormal_x_test_fft)

In [295]:
#SI Functions
def get_mean(x):
    avg = np.mean(x)
    return(avg)

def get_std(x):
    std = np.std(x)
    return(std)

def get_skewness(x):
    skewness = sum((x - np.mean(x))**3) / ((len(x)-1)*(np.std(x)**3))
    return(skewness)

def get_kurtosis(x):
    kurtosis = sum((x - np.mean(x))**4) / ((len(x)-1)*(np.std(x)**4))
    return (kurtosis)

def get_p2p(x):
    p2p = (np.max(x) - np.min(x))
    return(p2p)

def get_rms(x):
    rms = np.sqrt(sum(x**2) / len(x))
    return(rms)

def get_crestFactor(x):
    crestFactor = get_p2p(x) / get_rms(x)
    return(crestFactor)

def get_shapeFactor(x):
    shapeFactor = get_rms(x) / get_mean(x)
    return(shapeFactor)
    
def get_marginFactor(x):
    marginFactor = np.max(x) / (np.mean((np.sqrt(x)))**2)
    return(marginFactor)

def get_impulseFactor(x):
    impulseFactor = np.max(x) / np.mean(x)
    return(impulseFactor)


In [296]:
def make_df(x):
    mean_value = np.apply_along_axis(get_mean, 1, x)
    std_value = np.apply_along_axis(get_std, 1, x)
    skewness_value = np.apply_along_axis(get_skewness, 1, x)
    kurtosis_value = np.apply_along_axis(get_kurtosis, 1, x)
    p2p_value = np.apply_along_axis(get_p2p, 1, x)
    rms_value = np.apply_along_axis(get_rms, 1, x)
    crestFactor_value = np.apply_along_axis(get_crestFactor, 1, x)
    shapeFactor_value = np.apply_along_axis(get_shapeFactor, 1, x)
    marginFactor_value = np.apply_along_axis(get_marginFactor, 1, x)
    impulseFactor_value = np.apply_along_axis(get_impulseFactor, 1, x)
    
    df = pd.DataFrame({'mean':mean_value, 'std':std_value, 'skewness':skewness_value, 'kurtosis':kurtosis_value, 'p2p':p2p_value, 'rms':rms_value,
                       'crest':crestFactor_value, 'shape':shapeFactor_value, 'margin':marginFactor_value, 'impulse':impulseFactor_value})
    
    return(df)
    

In [297]:
#make dataframe
normal_x_train_SI = make_df(normal_x_train_cep)
normal_x_test_SI = make_df(normal_x_test_cep)

abnormal_x_train_SI = make_df(abnormal_x_train_cep)
abnormal_x_test_SI = make_df(abnormal_x_test_cep)

In [300]:
#Standardization in MTS
normal_x_train_SI_scaled = scale(normal_x_train_SI, axis = 0)
normal_x_test_SI_scaled = scale(normal_x_test_SI, axis = 0)

abnormal_x_train_SI_scaled = scale(abnormal_x_train_SI, axis = 0)
abnormal_x_test_SI_scaled = scale(abnormal_x_test_SI, axis = 0)

In [324]:
def get_MD_distance(x):
    normal_cov_mat = pd.DataFrame(normal_x_train_SI_scaled).corr()
    dist = np.matmul(np.matmul(x, inv(normal_cov_mat)), np.transpose(x)) / len(x)
    return(dist)

In [325]:
normal_train_dist = np.apply_along_axis(get_MD_distance, 1, normal_x_train_SI_scaled)
normal_test_dist = np.apply_along_axis(get_MD_distance, 1, normal_x_test_SI_scaled)

abnormal_train_dist = np.apply_along_axis(get_MD_distance, 1, abnormal_x_train_SI_scaled)
abnormal_test_dist = np.apply_along_axis(get_MD_distance, 1, abnormal_x_test_SI_scaled)


In [326]:
print(normal_train_dist)
print('\n')
print(normal_test_dist)
print('\n')
print(abnormal_train_dist)
print('\n')
print(abnormal_test_dist)


[0.99638816 1.20556823 0.6961474  1.03311977 1.25666036 1.11015733
 1.26563057 1.04936592 0.91568396 1.08046478 0.86949791 0.80369741
 0.68379644 0.83011264 1.20759038]


[ 2660.64730487 12397.26457582 22715.20821621   359.57995007
   246.61932123]


[   870.18914776   3881.10807099    832.71221819    651.88157729
   1295.21274734   1327.00415399   7825.5300613   16780.55689304
   1081.03492174   3007.06069686    256.36742725   2114.77331812
   1427.7928295   25177.09809296   2965.092063    15492.34922938
   4338.50435826   1822.98757764  14573.9532673    2690.54210415
   2850.82225455    870.37818962   3774.05975705   1804.71294789
   4251.5775226   18598.20719297  10312.19033427 137747.40669413
  20783.00022168  57242.78008809  27200.08370368  11381.63550656
  16991.45952569  61240.57894659   8136.66606203   1161.2570209
  55810.1302145    1635.170608     4206.14584024  22308.37485757
  17183.77328565    300.18346274    817.20941676 160768.8930934
   3658.60116289   2235.61985676   3

In [323]:
pd.DataFrame(normal_x_train_SI_scaled).corr()

,0,1,2,3,4,5,6,7,8,9
0,1.000000,0.980615,0.999245,0.909268,0.348488,0.945286,0.857138,0.932260,0.999050,0.857203
1,0.980615,1.000000,0.982705,0.961514,0.306591,0.947738,0.872978,0.985024,0.984310,0.873056
2,0.999245,0.982705,1.000000,0.911650,0.349906,0.947558,0.860811,0.936273,0.999949,0.860876
3,0.909268,0.961514,0.911650,1.000000,0.284891,0.908209,0.856100,0.977276,0.914173,0.856180
4,0.348488,0.306591,0.349906,0.284891,1.000000,0.594226,0.717601,0.258392,0.346228,0.717483
5,0.945286,0.947738,0.947558,0.908209,0.594226,1.000000,0.978246,0.919024,0.947683,0.978273
6,0.857138,0.872978,0.860811,0.856100,0.717601,0.978246,1.000000,0.858216,0.861112,1.000000
7,0.932260,0.985024,0.936273,0.977276,0.258392,0.919024,0.858216,1.000000,0.939401,0.858303
8,0.999050,0.984310,0.999949,0.914173,0.346228,0.947683,0.861112,0.939401,1.000000,0.861178
9,0.857203,0.873056,0.860876,0.856180,0.717483,0.978273,1.000000,0.858303,0.861178,1.000000


In [328]:
pd.DataFrame(inv(normal_cov_mat))

,0,1,2,3,4,5,6,7,8,9
0,4.576757e+05,-1.023237e+06,2.771156e+05,-1.263528e+03,-4.771989e+05,-3.333569e+05,1.609601e+09,4.323062e+05,-1.053379e+05,-1.608949e+09
1,-1.023238e+06,2.985702e+06,-6.545772e+05,2.829850e+03,9.101135e+05,-1.296506e+05,-3.862508e+09,-1.643272e+06,2.907274e+05,3.861974e+09
2,2.771156e+05,-6.545772e+05,3.318927e+05,-2.067099e+03,-2.597948e+05,-2.085051e+05,9.047750e+08,3.021229e+05,-2.114914e+05,-9.044118e+08
3,-1.263528e+03,2.829850e+03,-2.067099e+03,5.591543e+01,2.449077e+03,-1.334258e+02,-8.535857e+06,-1.112294e+03,2.057965e+03,8.533756e+06
4,-4.771989e+05,9.101135e+05,-2.597948e+05,2.449077e+03,6.486387e+05,4.356598e+05,-2.019342e+09,-2.755925e+05,1.205119e+05,2.018419e+09
5,-3.333569e+05,-1.296506e+05,-2.085051e+05,-1.334258e+02,4.356598e+05,1.479280e+06,-3.849963e+08,5.385870e+05,1.243281e+03,3.833518e+08
6,1.609601e+09,-3.862508e+09,9.047749e+08,-8.535856e+06,-2.019342e+09,-3.849964e+08,7.469099e+12,1.804677e+09,-4.952821e+08,-7.467175e+12
7,4.323062e+05,-1.643272e+06,3.021229e+05,-1.112294e+03,-2.755925e+05,5.385870e+05,1.804677e+09,1.105501e+06,-1.617575e+05,-1.805011e+09
8,-1.053379e+05,2.907274e+05,-2.114914e+05,2.057965e+03,1.205119e+05,1.243285e+03,-4.952821e+08,-1.617575e+05,1.793327e+05,4.951956e+08
9,-1.608949e+09,3.861974e+09,-9.044117e+08,8.533756e+06,2.018419e+09,3.833519e+08,-7.467175e+12,-1.805011e+09,4.951956e+08,7.465253e+12
